In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Define network structure

In [41]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Prepare data

In [42]:
# stable data
x_stable_1  = np.genfromtxt("input_stable_1000.csv", delimiter=',')
y_stable_1  = np.genfromtxt("output_stable_1000.csv", delimiter=',')

x_stable_2  = np.genfromtxt("input_stable_2000.csv", delimiter=',')
y_stable_2  = np.genfromtxt("output_stable_2000.csv", delimiter=',')

# unstable data
x_unstable_1  = np.genfromtxt("input_unstable_100.csv", delimiter=',')
y_unstable_1  = np.genfromtxt("output_unstable_100.csv", delimiter=',')

x_unstable_2  = np.genfromtxt("input_unstable_120.csv", delimiter=',')
y_unstable_2  = np.genfromtxt("output_unstable_120.csv", delimiter=',')

x_unstable_3  = np.genfromtxt("input_unstable_150.csv", delimiter=',')
y_unstable_3  = np.genfromtxt("output_unstable_150.csv", delimiter=',')

x_unstable_4  = np.genfromtxt("input_unstable_180.csv", delimiter=',')
y_unstable_4  = np.genfromtxt("output_unstable_180.csv", delimiter=',')

x_unstable_5  = np.genfromtxt("input_unstable_200.csv", delimiter=',')
y_unstable_5  = np.genfromtxt("output_unstable_200.csv", delimiter=',')

x_unstable_6  = np.genfromtxt("input_unstable_250.csv", delimiter=',')
y_unstable_6  = np.genfromtxt("output_unstable_250.csv", delimiter=',')

In [43]:
# From training data by concatenating data

# x_train = np.concatenate((x_stable_1, x_stable_2, x_unstable_1, x_unstable_2, x_unstable_3, x_unstable_4, x_unstable_5, x_unstable_6), axis=0)
# y_train = np.concatenate((y_stable_1, y_stable_2, y_unstable_1, y_unstable_2, y_unstable_3, y_unstable_4, y_unstable_5, y_unstable_6), axis=0)

x_train = np.concatenate((x_stable_2, x_unstable_1, x_unstable_2, x_unstable_3, x_unstable_4, x_unstable_5, x_unstable_6), axis=0)
y_train = np.concatenate((y_stable_2, y_unstable_1, y_unstable_2, y_unstable_3, y_unstable_4, y_unstable_5, y_unstable_6), axis=0)

x_train.shape, y_train.shape

((3000, 27), (3000, 3))

In [44]:
# abnormal data transformation
y_train[:,0][y_train[:,0] >  4]   =  4      # rocof_max (has been transformed to absolute value, Hz/s)
y_train[:,1][y_train[:,1] < -4]   = -4      # fnadir (Hz)
y_train[:,1][y_train[:,1] >  4]   =  4      # fnadir (Hz)
y_train[:,2][y_train[:,2] > 3.14] =  6      # maximum angle difference

# check transformation results
max(y_train[:,0]), min(y_train[:,0]), max(y_train[:,1]), min(y_train[:,1]), max(y_train[:,2]), min(y_train[:,2])

(4.0,
 0.045001259078598366,
 -0.07623773784599308,
 -4.0,
 6.0,
 0.03705011925890922)

In [45]:
# convert to torch tensor for training

x_train = x_train[:, 11:] # remove pg

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()

# Define training parameters

In [46]:
input_dim = 16
hidden_dim = 64
output_dim = 3

batch_size = 16

model = NeuralNetwork(input_dim, hidden_dim, output_dim)

dataset    = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [47]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

In [48]:
num_epochs = 3000  # Adjust as needed

for epoch in range(num_epochs):
    for batch_x, batch_y in dataloader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training loss
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/3000], Loss: 2.6311
Epoch [2/3000], Loss: 2.7644
Epoch [3/3000], Loss: 6.3183
Epoch [4/3000], Loss: 5.7213


Epoch [5/3000], Loss: 0.7647
Epoch [6/3000], Loss: 0.2034
Epoch [7/3000], Loss: 2.7598
Epoch [8/3000], Loss: 3.1140
Epoch [9/3000], Loss: 3.4557
Epoch [10/3000], Loss: 2.8871
Epoch [11/3000], Loss: 0.0844
Epoch [12/3000], Loss: 4.1512
Epoch [13/3000], Loss: 2.0644
Epoch [14/3000], Loss: 1.8578
Epoch [15/3000], Loss: 3.9929
Epoch [16/3000], Loss: 0.1981
Epoch [17/3000], Loss: 1.0490
Epoch [18/3000], Loss: 0.1628
Epoch [19/3000], Loss: 4.5680
Epoch [20/3000], Loss: 1.6567
Epoch [21/3000], Loss: 0.0734
Epoch [22/3000], Loss: 2.1057
Epoch [23/3000], Loss: 0.6614
Epoch [24/3000], Loss: 1.3769
Epoch [25/3000], Loss: 0.2057
Epoch [26/3000], Loss: 0.5977
Epoch [27/3000], Loss: 1.2610
Epoch [28/3000], Loss: 2.3112
Epoch [29/3000], Loss: 1.2078
Epoch [30/3000], Loss: 0.2598
Epoch [31/3000], Loss: 1.4580
Epoch [32/3000], Loss: 1.4165
Epoch [33/3000], Loss: 0.6700
Epoch [34/3000], Loss: 1.6470
Epoch [35/3000], Loss: 1.2254
Epoch [36/3000], Loss: 2.0288
Epoch [37/3000], Loss: 0.3651
Epoch [38/3000]

In [49]:
x1 = np.ones((16))*2
x1

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [50]:
x1 = torch.from_numpy(x1).float()
x1

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [51]:
model.forward(x1)

tensor([ 0.2947, -0.3281,  0.0557], grad_fn=<AddBackward0>)

# Save torch model

In [52]:
NN_name = f'NN_{input_dim}_{hidden_dim}_{output_dim}.pth'

torch.save(model.state_dict(), NN_name)